In [67]:
import glob
import sys
import logging
from tqdm import tqdm
from PIL import Image
from os import path
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time
import csv
import random
import pandas as pd
import math
import json

from tqdm.notebook import tqdm

In [14]:
input_dir = Path('../../data/01_raw/Hack the Planet')
coco_filepath = input_dir / "coco.json"
image_root_dir = input_dir / "images"

In [80]:
bear_images_root_dir = Path("../../data/05_model_input/beardetection/yolov8/test/images/")

In [82]:
def get_filepaths(root: Path, allowed_suffixes={".jpg", ".JPG", ".PNG"}) -> list[Path]:
    """Lists all filepaths given a root directory `root`."""
    return [p for p in root.rglob("*") if p.suffix in allowed_suffixes and p.exists()]

In [84]:
image_filepaths_with_bears = get_filepaths(root=bear_images_root_dir)

In [85]:
image_filepaths_with_bears

[PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/02190220.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00022b.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00019a.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00023b.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I__00012.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00018c.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00013a.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/02190222.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00018a.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00032a.JPG'),
 PosixPath('../../data/05_model_input/beardetection/yolov8/test/images/I_00017c.JPG'),
 PosixPath('../../data/05_model_input/beard

In [9]:
with open(coco_filepath, 'r') as f:
    data = json.load(f)

In [10]:
data.keys()

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [12]:
data["annotations"][0]

{'confidence': 0.993,
 'category_id': 1,
 'image_id': 0,
 'bbox': [655.872, 0.999972, 1263.936, 1048.896],
 'id': 0}

In [13]:
data["images"][0]

{'file_name': 'frameOutput/data/Season3 - animals only/22RucarAG/289_Draganu/20191014_20191202/CC0107/10180036_frames/image_180.jpg',
 'height': 1080,
 'width': 1920,
 'license_id': 1,
 'id': 0}

In [33]:
id_to_image_data = {image_data["id"]: image_data for image_data in data["images"]}

In [34]:
(list(id_to_image_data.items()))[:1]

[(0,
  {'file_name': 'frameOutput/data/Season3 - animals only/22RucarAG/289_Draganu/20191014_20191202/CC0107/10180036_frames/image_180.jpg',
   'height': 1080,
   'width': 1920,
   'license_id': 1,
   'id': 0})]

In [15]:
data["categories"]

[{'name': 'empty', 'supercategory': None, 'id': 0},
 {'name': 'animal', 'supercategory': None, 'id': 1},
 {'name': 'bear', 'supercategory': 'animal', 'id': 2}]

In [29]:
class_id_to_label = {category["id"]: category["name"] for category in data["categories"]}
label_to_class_id = {category["name"]: category["id"] for category in data["categories"]}

class_id_to_label, label_to_class_id

({0: 'empty', 1: 'animal', 2: 'bear'}, {'empty': 0, 'animal': 1, 'bear': 2})

In [16]:
annotations = data['annotations'][:10]

In [17]:
annotation = annotations[0]

In [18]:
annotation

{'confidence': 0.993,
 'category_id': 1,
 'image_id': 0,
 'bbox': [655.872, 0.999972, 1263.936, 1048.896],
 'id': 0}

In [37]:
image_data = id_to_image_data[annotation["image_id"]]
image_data, annotation

({'file_name': 'frameOutput/data/Season3 - animals only/22RucarAG/289_Draganu/20191014_20191202/CC0107/10180036_frames/image_180.jpg',
  'height': 1080,
  'width': 1920,
  'license_id': 1,
  'id': 0},
 {'confidence': 0.993,
  'category_id': 1,
  'image_id': 0,
  'bbox': [655.872, 0.999972, 1263.936, 1048.896],
  'id': 0})

In [40]:
label_to_class_id["bear"]

2

In [53]:
annotations_without_bears = [annotation for annotation in annotations if (annotation["category_id"] != label_to_class_id["bear"])]
image_filepaths_without_bears = [image_root_dir / id_to_image_data[annotation["image_id"]]["file_name"] for annotation in annotations_without_bears]

annotations_without_bears[0]
image_filepaths_without_bears[0]

PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Season3 - animals only/22RucarAG/289_Draganu/20191014_20191202/CC0107/10180036_frames/image_180.jpg')

In [56]:
image_filepath = image_filepaths_without_bears[1]
# Check if image_filepath exists
# Check if we can open the image_filepath
image = Image.open(image_filepath)

In [63]:
def is_valid_image_filepath(image_filepath: Path) -> bool:
    """
    Checks whether the image_filepath exists and can be read.
    """
    if not image_filepath.exists():
        return False
    else:
        try:
            image = Image.open(image_filepath)
            return True
        except:
            return False

In [59]:
is_valid_image_filepath(image_filepaths_without_bears[1])

True

In [62]:
is_valid_image_filepath(image_filepaths_without_bears[5])

True

In [64]:
image_filepaths_valid_without_bears = [image_filepath for image_filepath in image_filepaths_without_bears if is_valid_image_filepath(image_filepath)]

In [65]:
len(image_filepaths_valid_without_bears)

9

In [66]:
image_filepaths_valid_without_bears

[PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Season2 - animals only/21IzvoareleDamboviteiAG/161_Culmea Pecineagului/20181213_20190407/Bushn_CC0108/05130651_frames/image_240.jpg'),
 PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Bison - animals only/18RaulTarguluiAG/Exterior tarc/20211014_20211230/11120108_frames/image_90.jpg'),
 PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Season1 - animals only/POST 27/STRIMBISOARA_Bushnell_CC0YC/03280068_frames/image_180.jpg'),
 PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Bison - animals only/21IzvoareleDamboviteiAG/Bunea - curba/20201223_20210121/01050202_frames/image_60.jpg'),
 PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Season2 - animals only/22RucarAG/130_Fundu draganului/20181210_20190109/Bushn_CC00YF/12100031_frames/image_150.jpg'),
 PosixPath('../../data/01_raw/Hack the Planet/images/frameOutput/data/Season3 - animals only/2

In [70]:
def get_image_filepaths_without_bears(coco_filepath: dict, image_root_dir: Path) -> list[Path]:
    data = None
    with open(coco_filepath, 'r') as f:
        data = json.load(f)
    if not data:
        logging.error(f"could not load coco_filepath: {coco_filepath}")
        return []
    else:
        id_to_image_data = {image_data["id"]: image_data for image_data in data["images"]}
        class_id_to_label = {category["id"]: category["name"] for category in data["categories"]}
        label_to_class_id = {category["name"]: category["id"] for category in data["categories"]}
        # FIXME
        annotations = data['annotations']
        annotations_without_bears = [annotation for annotation in annotations if (annotation["category_id"] != label_to_class_id["bear"])]
        image_filepaths_without_bears = [image_root_dir / id_to_image_data[annotation["image_id"]]["file_name"] for annotation in annotations_without_bears]
        return [image_filepath for image_filepath in image_filepaths_without_bears if is_valid_image_filepath(image_filepath)]

In [71]:
image_filepaths_without_bears = get_image_filepaths_without_bears(coco_filepath=coco_filepath, image_root_dir=image_root_dir)

In [73]:
len(image_filepaths_without_bears)

14073

In [94]:
random_seed = 42
n_without_bears = len(image_filepaths_without_bears)
n_with_bears = len(image_filepaths_with_bears)
n = min(n_with_bears, n_without_bears)
n_threshold = 10


samples_without_bears = random.Random(random_seed).sample(image_filepaths_without_bears, min(n, n_threshold))
samples_with_bears = random.Random(random_seed).sample(image_filepaths_with_bears, min(n, n_threshold))

In [93]:
n_without_bears, n_with_bears, n_threshold, n, len(samples_with_bears), len(samples_without_bears)

(14073, 44, 20, 44, 20, 20)

In [95]:
model_weights_filepath = Path("../../data/06_models/beardetection/yolov8/best/weights/best.pt") 

In [96]:
from ultralytics import YOLO

In [97]:
model = YOLO(model_weights_filepath)

In [98]:
model.info()

Model summary: 225 layers, 3011043 parameters, 0 gradients, 8.2 GFLOPs


(225, 3011043, 0, 8.1941504)

In [109]:
model.predict(samples_with_bears[0], save=True)


image 1/1 /home/chouffe/fruitpunch/challenges/bear/notebooks/bearedge/../../data/05_model_input/beardetection/yolov8/test/images/03080008.JPG: 768x1024 1 bearbody, 173.8ms
Speed: 11.6ms preprocess, 173.8ms inference, 0.9ms postprocess per image at shape (1, 3, 768, 1024)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bearbody'}
 obb: None
 orig_img: array([[[251, 242, 238],
         [251, 242, 238],
         [251, 242, 238],
         ...,
         [255, 253, 253],
         [255, 254, 254],
         [255, 255, 255]],
 
        [[252, 243, 239],
         [251, 242, 238],
         [250, 241, 237],
         ...,
         [255, 253, 253],
         [255, 254, 254],
         [255, 255, 255]],
 
        [[250, 241, 237],
         [249, 240, 236],
         [249, 240, 236],
         ...,
         [255, 255, 255],
         [252, 250, 250],
         [248, 246, 246]],
 
        ...,
 
        [[ 39,  85, 196],
         [ 49,  94, 209],
         [ 37,  78, 200],
         ...,
         [252, 251, 255],
         [252, 251, 255],
         [252, 251, 255]],
 
        [[ 35,  81, 192],
         [ 37,  82, 197],
         [ 58,  99, 221],
         ...,
         [255, 2

In [101]:
model.predict(samples_with_bears[0])


image 1/1 /home/chouffe/fruitpunch/challenges/bear/notebooks/bearedge/../../data/05_model_input/beardetection/yolov8/test/images/03080008.JPG: 768x1024 1 bearbody, 185.7ms
Speed: 10.2ms preprocess, 185.7ms inference, 6.0ms postprocess per image at shape (1, 3, 768, 1024)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bearbody'}
 obb: None
 orig_img: array([[[251, 242, 238],
         [251, 242, 238],
         [251, 242, 238],
         ...,
         [255, 253, 253],
         [255, 254, 254],
         [255, 255, 255]],
 
        [[252, 243, 239],
         [251, 242, 238],
         [250, 241, 237],
         ...,
         [255, 253, 253],
         [255, 254, 254],
         [255, 255, 255]],
 
        [[250, 241, 237],
         [249, 240, 236],
         [249, 240, 236],
         ...,
         [255, 255, 255],
         [252, 250, 250],
         [248, 246, 246]],
 
        ...,
 
        [[ 39,  85, 196],
         [ 49,  94, 209],
         [ 37,  78, 200],
         ...,
         [252, 251, 255],
         [252, 251, 255],
         [252, 251, 255]],
 
        [[ 35,  81, 192],
         [ 37,  82, 197],
         [ 58,  99, 221],
         ...,
         [255, 2

In [115]:
model.predict(samples_without_bears[5], save=True)


image 1/1 /home/chouffe/fruitpunch/challenges/bear/notebooks/bearedge/../../data/01_raw/Hack the Planet/images/frameOutput/data/Season1 - animals only/POST 28/BLIDARU_Bushnell_CC00YY_FCC-CAM-022/03260425_frames/image_240.jpg: 576x1024 1 bearbody, 127.0ms
Speed: 4.9ms preprocess, 127.0ms inference, 0.8ms postprocess per image at shape (1, 3, 576, 1024)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bearbody'}
 obb: None
 orig_img: array([[[176, 168, 168],
         [176, 168, 168],
         [176, 168, 168],
         ...,
         [103, 115, 125],
         [ 63,  79,  92],
         [ 47,  64,  77]],
 
        [[176, 168, 168],
         [176, 168, 168],
         [176, 168, 168],
         ...,
         [111, 123, 133],
         [ 72,  88, 100],
         [ 48,  65,  78]],
 
        [[176, 168, 168],
         [176, 168, 168],
         [176, 168, 168],
         ...,
         [124, 136, 146],
         [ 87, 105, 116],
         [ 53,  72,  85]],
 
        ...,
 
        [[ 49,  96, 187],
         [145, 182, 250],
         [190, 204, 232],
         ...,
         [250, 250, 250],
         [250, 250, 250],
         [250, 250, 250]],
 
        [[ 49,  96, 187],
         [145, 182, 250],
         [190, 204, 232],
         ...,
         [250, 2

In [103]:
yolov8_predictions = model.predict(samples_without_bears[0])


image 1/1 /home/chouffe/fruitpunch/challenges/bear/notebooks/bearedge/../../data/01_raw/Hack the Planet/images/frameOutput/data/Bison - animals only/18RaulTarguluiAG/Hranitor/20210226_20210308/03070944_frames/image_570.jpg: 576x1024 2 bearbodys, 118.8ms
Speed: 7.3ms preprocess, 118.8ms inference, 0.8ms postprocess per image at shape (1, 3, 576, 1024)


In [107]:
yolov8_predictions[0]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'bearbody'}
obb: None
orig_img: array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 18,  10,  20],
        [ 26,  19,  26],
        [ 32,  28,  34]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 27,  20,  27],
        [ 22,  18,  24],
        [ 23,  19,  25]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 29,  22,  29],
        [ 23,  19,  25],
        [ 27,  23,  28]],

       ...,

       [[  0, 105, 255],
        [  0, 105, 255],
        [  0, 105, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[  0, 105, 255],
        [  0, 105, 255],
        [  0, 105, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [25

In [ ]:
yolov8_predictions